# 第５章　文章の穴埋め

文章の穴埋めを理解する

BERTを用いた文章の穴埋めを実装する

In [ ]:
# -*- coding:utf-8 -*-

In [ ]:
# ライブラリのインストール
!pip install transformers
!pip install fugashi
!pip install ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 31.4 MB/s 
     |████████████████████████████████| 596 kB 52.9 MB/s 
     |████████████████████████████████| 101 kB 14.5 MB/s 
     |████████████████████████████████| 6.6 MB 58.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 568 kB 35.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.4 MB 33.8 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=9483c4fc4c462b6bad73ba6ecd1b49ad601015ede61b0a619a815280e0a450ea
  Stored in directory: /root/.cache/pip/wheels/33/8b

In [ ]:
# ライブラリのインポート
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

In [ ]:
# トークナイザとモデルのダウンロード
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'   # 日本語事前学習済みモデル　東北大より
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)   
print(tokenizer)

PreTrainedTokenizer(name_or_path='cl-tohoku/bert-base-japanese-whole-word-masking', vocab_size=32000, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [ ]:
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()  # GPU対応　これがColaboratoryでしかできない
print(bert_mlm)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
# テキストのトークン化
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


出力の属性 logitsとして、語彙に含まれる各トークンの分類スコアを表すテンソル scores が得られる。scores は３次元配列. サイズは（バッチサイズ、系列長、語彙のサイズ）

## BertForMaskedLMがどのように実装されているのか
BertModelから得られる最終レイヤーの出力に対して、線形変換、GELU関数、線形変換を適用して、分類スコアを出している。（正規化、正則化の層は除く）

## score から [MASK] に入るトークンを予測するには

i番目の文章のマスクを穴埋めする。まずはトークン列においてどこがMASKなのかを調べる。MASKのトークンIDは4(トークンID：トークンに割り振られるID）scores[i, j]はサイズ32000（語彙のサイズ）の１次元配列。各要素がMASKに対する分類スコアを出している、つまり32,000の中から分類スコアが高いものを抽出して当てはめると適切な語彙が入れられるのではないかということ。

In [ ]:
# 文章を符号化し、GPUに配置する
input_ids = tokenizer.encode(text, return_tensors='pt')  # GPUに配置とは。返り値をテンソル型にする
input_ids = input_ids.cuda()   # ここでGPUに配置している

# BERTに入力し、分類スコアを得る
# 系列長を揃える必要がないので、単にinput_idsのみを入力する
with torch.no_grad():  # no_gradとは with文と併用しているが機能がわからない  4章最後
  output = bert_mlm(input_ids=input_ids)
  scores = output.logits  # 出力の属性 logits: テンソル scores が得られる（３次元配列）

In [ ]:
print(input_ids[0].tolist().index(4)) # input_idsのshapeは1行8列これをリストに変換し、4=MASKの要素を要素番号を出力する
print(input_ids.shape)

3
torch.Size([1, 8])


In [ ]:
# ID列で[MASK]の位置を調べる
mask_position = input_ids[0].tolist().index(4)  # mask_position: 文章中のMASKの位置情報

# スコアが最も良いトークンのIDを取り出し、トークンに変換する
print(scores.shape)  # 1行バッチサイズ、8のトークン系列長、32000の語彙のサイズ
id_best = scores[0, mask_position].argmax(-1).item()
print(id_best)  # id_bestの中身を確認 391が選ばれた32000の中から
token_best = tokenizer.convert_ids_to_tokens(id_best)  # 391→東京
print(token_best)# token_bestの中身を確認
token_best = token_best.replace('##', '')

# [MASK]を上で求めたトークンで置き換える
text = text.replace('[MASK]', token_best)

print(text)

torch.Size([1, 8, 32000])
391
東京
今日は東京へ行く。


In [ ]:
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
  """
  文章中の最初の[MASK]をスコアの上位のトークンに置き換える
  上位何位まで使うかは、num_topkで指定
  出力は穴埋めされた文章のリストと、置き換えられたトークンのスコアのリスト
  """

  # 文章を符号化し、BERTで分類スコアを得る
  input_ids = tokenizer.encode(text, return_tensors='pt')  # テキストを入れて返り値はテンソル型
  input_ids = input_ids.cuda()  # GPU対応
  with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
  scores = output.logits  # (バッチサイズ、系列長、語彙のサイズ)

  # スコアが上位のトークンとスコアを求める
  mask_positiion = input_ids[0].tolist().index(4)  # 先のコードと同じ
  topk = scores[0, mask_position].topk(num_topk)   # num_topk  10個の値がでる、先の391のように6166, 466, 1724, 5359, 2118, 286, 2030, 1221, 2249
  print(topk)  # topk の内容確認
  ids_topk = topk.indices # トークンのID
  print("ここからids_topk", ids_topk)  # ids_topk を確認
  tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk) # 実際の語彙が出る 以下のfor文のイテレータ
  print("ここからtokens_topk",tokens_topk)
  scores_topk = topk.values.cpu().numpy() # スコア
  print("ここからscores_topk", scores_topk)

  # 文章中の[MASK]を上で求めたトークンで置き換える
  text_topk = [] # 穴埋めされたテキストを追加する
  for token in tokens_topk:
    token = token.replace('##', '')
    # リストに完成した文を追加 [MASK]は置き換えられる
    text_topk.append(text.replace('[MASK]', token, 1))  # 完全一致か部分か

  return text_topk, scores_topk


text = '企業は[MASK]です。'
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm, 10)   
print("返り値の確認", text_topk)
print(*text_topk, sep='\n')  # おそらくすべてのtext_topkを出力する命令, 行末を改行に指定

torch.return_types.topk(
values=tensor([12.2410,  9.5000,  9.2430,  9.1664,  8.9356,  8.9081,  8.7873,  8.6166,
         8.4883,  8.1098], device='cuda:0'),
indices=tensor([13564, 21005,  1520,  1275,   692,  3441,     6,  9156,  1195,  4691],
       device='cuda:0'))
ここからids_topk tensor([13564, 21005,  1520,  1275,   692,  3441,     6,  9156,  1195,  4691],
       device='cuda:0')
ここからtokens_topk ['太字', '非公開', 'すべて', '株式会社', '別', '一覧', '、', '全部', '全て', '無料']
ここからscores_topk [12.24096    9.499981   9.243002   9.166376   8.935606   8.908144
  8.787287   8.616574   8.48833    8.1097555]
返り値の確認 ['企業は太字です。', '企業は非公開です。', '企業はすべてです。', '企業は株式会社です。', '企業は別です。', '企業は一覧です。', '企業は、です。', '企業は全部です。', '企業は全てです。', '企業は無料です。']
企業は太字です。
企業は非公開です。
企業はすべてです。
企業は株式会社です。
企業は別です。
企業は一覧です。
企業は、です。
企業は全部です。
企業は全てです。
企業は無料です。


## 貪欲法
仮にMASKが2つ存在する状況を仮定すると、合計で32000**2の組み合わせの候補が存在する。

これをすべて調べることはコストが高いので、近似的な方法で代替する。方法としては最初のMASKを最も高いスコアの語彙に置き換える。残りのMASKは1つ目が置き換えられた文章を使って最も高いスコアの語彙に置き換える。

In [ ]:
# 貪欲法
def greeby_prediction(text, tokenizer, bert_mlm):
  """
  [MASK]を含む文章を入力として、貪欲法で穴埋めを行った文章を出力する
  """
  # 前から順に[MASK]を一つずつ、スコアの最も高いトークンに置き換える
  for _ in range(text.count('[MASK]')):  # for 文を回す階数はMASKの数
    text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
  return text

text = '今日は[MASK][MASK]へ行く。'
greeby_prediction(text, tokenizer, bert_mlm)

# うまく出力されていない、参考書は今日は東京へ行くと出力されて違和感がない

torch.return_types.topk(
values=tensor([7.1444], device='cuda:0'),
indices=tensor([6], device='cuda:0'))
ここからids_topk tensor([6], device='cuda:0')
ここからtokens_topk ['、']
ここからscores_topk [7.1443725]
torch.return_types.topk(
values=tensor([20.7564], device='cuda:0'),
indices=tensor([6], device='cuda:0'))
ここからids_topk tensor([6], device='cuda:0')
ここからtokens_topk ['、']
ここからscores_topk [20.756357]


'今日は、、へ行く。'

## ビームサーチ

BERTは文章を前から順番に生成するというような、自然言語処理でよくある文章生成は得意ではない。BERTは周りの文脈からもとのトークンを予測するというタスクを用いており、大部分がMASKトークンになっているようなものを学習していない。貪欲法の他により性能の良い近似手法としてビームサーチと呼ばれる方法がある。

ビームサーチは1つ目のMASKを例えばスコアが上位10のトークンで置き換えた10の文章を作成する。次に得られた10の文章それぞれに対して次のMASKを同じく上位10のトークンで置き換えた10の文章を作る。合計100個の文章を選び出す。この中から合計スコアの高い上位10の文章を選び出す。

In [ ]:
# ビームサーチの実装

def beam_search(text, tokenizer, bert_mlm, num_topk):
  """ ビームサーチで文章の穴埋めを行う """
  num_mask = text.count('[MASK]')  # [MASK]トークンの数を数える
  text_topk = [text]  # テキストをリスト化 len() = 1
  scores_topk = np.array([0])
  for _ in range(num_mask):  # [MASK]トークンの数だけ回す
      # 現在得られている、それぞれの文章に対して、最初の[MASK]をスコアが上位のトークンで穴埋めする
      text_candidates = []  # それぞれの文章を穴埋めした結果を追加する
      score_candidates = []  # 穴埋めに使ったトークンのスコアを追加する
      for text_mask, score in zip(text_topk, scores_topk):
          text_topk_inner, scores_topk_inner = predict_mask_topk(  # 関数の返り値を代入
              text_mask, tokenizer, bert_mlm, num_topk
          )
          text_candidates.extend(text_topk_inner)
          print(text_candidates)
          score_candidates.append( score + scores_topk_inner )
          print(score_candidates)
      
      # 穴埋めにより生成された文章の中から合計スコアの高いものを選ぶ
      score_candidates = np.hstack(score_candidates)
      print("hstack", score_candidates)
      idx_list = score_candidates.argsort()[::-1][:num_topk]
      print(idx_list)
      text_topk = [ text_candidates[idx] for idx in idx_list ]
      scores_topk = score_candidates[idx_list]

  return text_topk

text = "今日は[MASK][MASK]へ行く。"
print(tokenizer.tokenize(text))
print([text])
text_topk = [text]
print(type(text_topk))
print(len(text_topk))

text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(text_topk)

# こちらもうまく出力しなかった。

['今日', 'は', '[MASK]', '[MASK]', 'へ', '行く', '。']
['今日は[MASK][MASK]へ行く。']
<class 'list'>
1
torch.return_types.topk(
values=tensor([7.1444, 6.6560, 5.8840, 5.3418, 5.3251, 5.2564, 5.2122, 5.0689, 5.0311,
        5.0203], device='cuda:0'),
indices=tensor([   6, 1438,   59, 7755,  552,   73,   70, 1842,  733,  391],
       device='cuda:0'))
ここからids_topk tensor([   6, 1438,   59, 7755,  552,   73,   70, 1842,  733,  391],
       device='cuda:0')
ここからtokens_topk ['、', '再び', 'その', 'あの', '同じ', 'お', 'この', '新しい', 'ゲーム', '東京']
ここからscores_topk [7.1443725 6.6560054 5.8840456 5.341809  5.3250957 5.2564464 5.2121897
 5.068879  5.031147  5.0202594]
['今日は、[MASK]へ行く。', '今日は再び[MASK]へ行く。', '今日はその[MASK]へ行く。', '今日はあの[MASK]へ行く。', '今日は同じ[MASK]へ行く。', '今日はお[MASK]へ行く。', '今日はこの[MASK]へ行く。', '今日は新しい[MASK]へ行く。', '今日はゲーム[MASK]へ行く。', '今日は東京[MASK]へ行く。']
[array([7.1443725, 6.6560054, 5.8840456, 5.341809 , 5.3250957, 5.2564464,
       5.2121897, 5.068879 , 5.031147 , 5.0202594], dtype=float32)]
hstack [7.1443725 6.6560054